Investment strategy brainstorm
- ROI: begin with 100 dollars and see how it goes

In [20]:
from firebase import firebase
import json
import pandas as pd
import numpy as np
from collections import deque

import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression


from datetime import timedelta
import datetime
#import missingno as msno
import tensorflow as tf
import IPython 
import IPython.display

In [4]:
my_firebase = firebase.FirebaseApplication('https://test-random-305921-default-rtdb.firebaseio.com/', None)

In [7]:
trades=my_firebase.get('/websocket_trades_v1_2021-03-12', '')
df=pd.DataFrame(trades).T.reset_index(drop=True)

In [18]:
print(df.shape)
df.head()

(78021, 16)


,best_ask,best_bid,high_24h,last_size,low_24h,open_24h,price,product_id,sequence,side,time,trade_id,type,volume_24h,volume_30d,bid_ask_spread
0,57423.23,57419.99,58113,0.002903,55050,57527.51,57423.23,BTC-USD,22637372049,buy,2021-03-12 18:51:21.986589+00:00,144106022,ticker,23239.10554988,728721.92359102,3.24
1,57424.56,57424.55,58113,0.006150,55050,57527.51,57424.55,BTC-USD,22637372497,sell,2021-03-12 18:51:22.875002+00:00,144106023,ticker,23239.11169988,728721.92974102,0.01
2,57425.61,57424.55,58113,0.001096,55050,57527.51,57424.56,BTC-USD,22637372583,buy,2021-03-12 18:51:23.158075+00:00,144106024,ticker,23239.11279604,728721.93083718,1.06
3,57425.61,57424.55,58113,0.003173,55050,57527.51,57425.61,BTC-USD,22637372585,buy,2021-03-12 18:51:23.158075+00:00,144106025,ticker,23239.11596886,728721.93401000,1.06
4,57426.73,57425.60,58113,0.006327,55050,57527.51,57425.61,BTC-USD,22637372772,buy,2021-03-12 18:51:23.662085+00:00,144106026,ticker,23239.12229604,728721.94033718,1.13


In [14]:
#datatypes
df.time=pd.to_datetime(df.time)
df["price"] = df.price.astype(float)
df["last_size"] = df['last_size'].astype(float)
df["best_bid"] = df['best_bid'].astype(float)
df["best_ask"] = df['best_ask'].astype(float)

In [16]:
df['bid_ask_spread']=df['best_ask']-df['best_bid']

In [ ]:
for i in range(1000, 10000):
